In [ ]:
import pickle
from detectors import *
import matplotlib.pyplot as plt
import numpy as np
import random

# load data

In [ ]:
with open('data/twitter/biden_embeddings.pickle', 'rb') as handle:
    biden = pickle.load(handle)
with open('data/twitter/trump_embeddings.pickle', 'rb') as handle:
    trump = pickle.load(handle)

In [ ]:
random.shuffle(biden)
random.shuffle(trump)

chunk_size = 1000

biden_chunks = [[list(x) for x in zip(*biden[i:i + chunk_size])][1] for i in range(0, len(biden), chunk_size)]
trump_chunks = [[list(x) for x in zip(*trump[i:i + chunk_size])][1] for i in range(0, len(trump), chunk_size)]

# initiate detectors

In [ ]:
detectors = {}

csdd = CosineSimilarityDriftDetector()
detectors['csdd'] = csdd

kts = KernelTwoSampleDriftDetector()
detectors['kts'] = kts

aks = AlibiKSDetector()
detectors['aks'] = aks

acs = AlibiChiSquaredDetector()
detectors['acs'] = acs

ammd = AlibiMMDDetector()
detectors['ammd'] = ammd

# tests

In [ ]:
results = {x: [] for x in detectors}


for i in range(20):
    for detector in detectors:
        detectors[detector].fit(np.array(biden_chunks[i]))
        results[detector].append(detectors[detector].predict_proba(np.array(trump_chunks[i])))

# plot graphs

In [ ]:
for result in results:
    plt.plot(results[result])
    #plt.xticks(ticks=np.arange(len(target_percentages)), labels=target_percentages, rotation=60)
    plt.savefig('figures/by_dataset/twitter_bert_768_different_classes/{}.pdf'.format(result), format='pdf')
    plt.show()
    
for result in results:
    plt.plot(results[result])

#plt.xticks(ticks=np.arange(len(target_percentages)), labels=target_percentages, rotation=60)
plt.savefig('figures/by_dataset/twitter_bert_768_different_classes/{}.pdf'.format('all'), format='pdf')
plt.show()

In [ ]:
print(results)